In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import sys
import pandas as pd
import numpy as np
import scipy.stats
import math
#import seaborn as sns
from sets import Set

/home/linuxbrew/.linuxbrew/lib/python2.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: the sets module is deprecated


In [2]:
def filterValues(colname, DF, val):
    DF.loc[DF[colname] < val, colname] = 0.0

def relError(c1, c2, DF, cutoff=0.00999999, verbose=False):
    import pandas as pd
    import numpy as np
    nz = DF[DF[c1] > cutoff]
    re = (nz[c1] - nz[c2]) / nz[c1]
    return re

def proportionalityCorrelation(c1, c2, DF, offset=0.01):
    import numpy as np
    return (2.0 * np.log(DF[c1] + offset).cov(np.log(DF[c2] + offset))) / (np.log(DF[c1] + offset).var() + np.log(DF[c2] + offset).var())

def relDiffTP(c1, c2, DF, cutoff=0.1):
    import pandas as pd
    import numpy as np
    tpindex = DF[DF[c1] > cutoff]
    rd = (tpindex[c2] - tpindex[c1]) / tpindex[c1]
    return rd

def getMedian(df): return df.mean()
def getMean(df): return df.mean()

def relDiff(c1, c2, DF, cutoff=0.01, verbose=False):
    import pandas as pd
    """
    Computes the relative difference between the values
    in columns c1 and c2 of DF.
    c1 and c2 are column names and DF is a Pandas data frame.
    Values less than cutoff will be treated as 0.
    The relative difference is defined as
    d(x_i, y_i) =
        0.0 if x_i and y_i are 0
        (x_i - y_i) / (0.5 * |x_i - y_i|) otherwise
    This function returns two values.
    rd is a DataFrame where the "relDiff" column contains all
    of the computed relative differences.
    nz is a set of indexes into rd for data points where
    x_i and y_i were not *both* zero.
    """
    import numpy as np
    rd = pd.DataFrame(data = {"Name" : DF.index, "relDiff" : np.zeros(len(DF.index))*np.nan})
    rd.set_index("Name", inplace=True)
    bothZero = DF.loc[(DF[c1] < cutoff) & (DF[c2] < cutoff)].index
    nonZero = DF.index.difference(bothZero)
    if (verbose):
        print("Zero occurs in both columns {} times".format(len(rd.loc[bothZero])))
        print("Nonzero occurs in at least 1 column {} times".format(len(rd.loc[nonZero])))
    allRD = 1.0 * ((DF[c1] - DF[c2]) / (DF[c1] + DF[c2]).abs())
    assert(len(rd.loc[nonZero]["relDiff"]) == len(allRD[nonZero]))
    rd["relDiff"][nonZero] = allRD[nonZero]
    if len(bothZero) > 0:
        rd["relDiff"][bothZero] = 0.0
    return rd, nonZero

In [3]:
cutoff = 1
tpm = sys.argv[0]
#data_dir = sys.argv[1]
data_dir = "/mnt/scratch6/selective_alignment_data/"
ground_truth_file = "/mnt/scratch6/selective_alignment_data/rsem/rsem_sim/sim.sim.isoforms.results"
rsem_quant_file = "/mnt/scratch6/selective_alignment_data/rsem/rsem_exp/model.isoforms.results"
salmon_quant_file = "/mnt/scratch6/selective_alignment_data/salmon_out/quant.sf"
kallisto_quant_file = "/mnt/scratch6/selective_alignment_data/kallisto_result/abundance.tsv"
hera_quant_file = "/mnt/scratch6/selective_alignment_data/hera_out/abundance.tsv"

In [4]:
ans = pd.read_table(ground_truth_file,index_col="transcript_id")
ans = ans.rename(columns = {'count':'count1'})

In [12]:
rsem = pd.read_table(rsem_quant_file,index_col="transcript_id")
ans = ans.rename(columns = {'effective_length':'efflen1'})
#join_ans_rsem = pd.concat([ans,rsem],axis=1,join="inner")
join_ans_rsem = rsem.join(ans,rsuffix="_truth").fillna(0.0)
##print(len(join_ans_rsem))
rd, nonZero =  relDiff('expected_count', 'count1', join_ans_rsem, cutoff)
rd['relDiff'] = rd['relDiff'].abs()
##print np.sum(rsem['expected_count'])
MARD_rsem = np.mean(rd['relDiff'].abs())
spearman_rsem = scipy.stats.spearmanr(join_ans_rsem['expected_count'],join_ans_rsem['count1']).correlation
print("RSEM MARD: {}".format(MARD_rsem))
print("RSEM SPEARMAN: {}".format(spearman_rsem))

RSEM MARD: 0.379724588824
RSEM SPEARMAN: 0.979024410662


In [19]:
hera = pd.read_table(hera_quant_file,names=["target_id","unique_map","length","eff_length","est_counts","tpm"],header=None,skiprows=1)
b = hera['target_id'].str.split(":")
hera['id'] = b
df = pd.concat([hera,pd.DataFrame(hera['id'].tolist())], axis=1, join='outer')
hera = df.set_index(keys=df[0])
join_ans_hera = hera.join(ans,rsuffix="_truth").fillna(0.0)
rd, nonZero =  relDiff('est_counts', 'count1', join_ans_hera, cutoff)
MARD_hera = np.mean(rd['relDiff'].abs())
spearman_hera = scipy.stats.spearmanr(join_ans_hera['est_counts'],join_ans_hera['count1']).correlation
print("HERA MARD: {}".format(MARD_hera))
print("HERA SPEARMAN: {}".format(spearman_hera))

HERA MARD: 0.0637891789539
HERA SPEARMAN: 0.84080108528


In [22]:
kallisto = pd.read_table(kallisto_quant_file,index_col="target_id")
join_ans_kallisto = kallisto.join(ans,rsuffix="_truth").fillna(0.0)
rd, nonZero =  relDiff('est_counts', 'count1', join_ans_kallisto, cutoff)
MARD_kallisto = np.mean(rd['relDiff'].abs())
spearman_kallisto = scipy.stats.spearmanr(join_ans_kallisto['est_counts'],join_ans_kallisto['count1']).correlation
print("KALLISTO MARD: {}".format(MARD_kallisto))
print("KALLISTO SPEARMAN: {}".format(spearman_kallisto))

KALLISTO MARD: 0.0745809814224
KALLISTO SPEARMAN: 0.810008827719


In [23]:
salmon = pd.read_table(salmon_quant_file,index_col="Name")
join_ans_salmon = salmon.join(ans,rsuffix="_truth").fillna(0.0)
rd, nonZero =  relDiff('NumReads', 'count1', join_ans_salmon, cutoff)
MARD_salmon = np.mean(rd['relDiff'].abs())
spearman_salmon = scipy.stats.spearmanr(join_ans_salmon['NumReads'],join_ans_salmon['count1']).correlation
print("SALMON_SL MARD: {}".format(MARD_salmon))
print("SALMON_SL SPEARMAN: {}".format(spearman_salmon))

SALMON_SL MARD: 0.0569800996206
SALMON_SL SPEARMAN: 0.846416491037
